In [5]:
# ist736 text mining hw_1 - lexicon-based analysis
# dujun zhai

# task 1: find the top three social relationships mentioned in happy moments


# step 1: load datasets
# ================================

# bringing in pandas, need it to handle csv files
import pandas as pd  

# happydb dataset, contains happy moments written by people
# this is the main dataset we'll analyze
happydb_url = "https://raw.githubusercontent.com/megagonlabs/HappyDB/master/happydb/data/cleaned_hm.csv"
happydb = pd.read_csv(happydb_url)

# people dictionary, contains words that describe social relationships
# we need this to identify which relationships appear in the happy moments
people_dict_url = "https://raw.githubusercontent.com/megagonlabs/HappyDB/blob/master/happydb/data/topic_dict/people-dict.csv"
people_dict = pd.read_csv(people_dict_url)


# step 2: check if data loaded correctly
# ================================

# print first few rows, make sure datasets are structured as expected
print("happydb sample:", happydb.head(), "\n")  
print("people dictionary sample:", people_dict.head(), "\n")  

# reasoning:
# if the data isn't loaded properly, everything else will be wrong
# checking now avoids errors later, better to fix issues early


# step 3: extract relationship words
# ================================

# pulling out unique relationship words from people dictionary
# each word represents a type of social connection
relationship_words = people_dict['category'].unique()  

# reasoning:
# these are the words we'll look for in the happy moments
# if a word is in the list, that means it counts as a relationship mention

# print extracted relationship words, quick verification
print("unique relationship words:", relationship_words, "\n")

# reasoning:
# better to check now than to debug later
# knowing exactly what words we are searching for helps avoid mistakes


HTTPError: HTTP Error 404: Not Found

In [6]:
import requests

# URL of the file
url = "https://raw.githubusercontent.com/megagonlabs/HappyDB/master/happydb/data/cleaned_hm.csv"

# Send an HTTP request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the content to a file
    with open("cleaned_hm.csv", "wb") as file:
        file.write(response.content)
    print("Download complete: cleaned_hm.csv")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


Download complete: cleaned_hm.csv
